# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [ ]:
import tensorflow as tf

# Print TensorFlow version for reference
print("TensorFlow version:", tf.__version__)

# List available GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs detected:", gpus)
    # Optional: Enable memory growth to avoid allocating all GPU memory at once
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU detected, using CPU.")


In [ ]:
import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

In [ ]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from tensorflow import random

In [ ]:
import sys
print("Python executable:", sys.executable)

In [ ]:
# Cell 1: Set environment variable before any TensorFlow import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Ensure GPU 0 is visible

# Cell 2: Import TensorFlow and verify GPU setup
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs available:", tf.config.list_physical_devices('GPU'))

# Continue with your model definition and training...

In [ ]:
print(tf.sysconfig.get_build_info())

### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [ ]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [ ]:
classifier.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))

### Step 4: Now, add the pooling layer with the image size as 2x2

In [ ]:
classifier.add(MaxPool2D(pool_size=(2, 2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [ ]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [ ]:
classifier.add(Dense(128, activation='relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [ ]:
classifier.add(Dense(1, activation='sigmoid'))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directory where your images are stored
data_dir = '/home/oem/Downloads/train'  # Update this path if needed

# List all image files (assuming they end with '.jpg')
files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Create a DataFrame with filenames and labels
labels = []
for file in files:
    if file.startswith('cat'):
        labels.append('cat')
    elif file.startswith('dog'):
        labels.append('dog')
    else:
        labels.append('unknown')  # In case there are files that don't match

df = pd.DataFrame({'filename': files, 'class': labels})

# Optionally, you can filter out any unknown labels
df = df[df['class'] != 'unknown']

# Split the DataFrame into training (90%) and test (10%) sets
train_df, test_df = train_test_split(
    df, 
    test_size=0.1, 
    random_state=1,   # or any seed you prefer
    stratify=df['class']  # ensures proportional split per class
)

# Create ImageDataGenerators with your desired settings
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

In [ ]:
# Create generators using the DataFrame
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=data_dir,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'  # since it's a binary classification
)

### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=data_dir,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.

In [ ]:
# Calculate steps per epoch for training and testing
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_TEST  = test_generator.n // test_generator.batch_size

# Fit the model using the generators
history = classifier.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=30,  # set the number of epochs as needed
    validation_data=test_generator,
    validation_steps=STEP_SIZE_TEST
)